___

<a href='https://github.com/eliasmelul/'> <img src='https://s3.us-east-2.amazonaws.com/wordontheamazon.com/NoMargin_NewLogo.png' style='width: 15em;' align='right' /></a>
# Finding my Schitt's Creek
#### Modeling
___
<h3 align="right">by Elias Melul, Data Scientist </h3> 

___



In [1]:
import pandas as pd # import pandas for dataframes
import numpy as np
import requests
from bs4 import BeautifulSoup
import locale
from datetime import datetime
import re
import matplotlib.pyplot as plt
import seaborn as sns
import json
import folium
from pandas.io.json import json_normalize


import matplotlib.cm as cm
import matplotlib.colors as colors

from IPython.display import HTML, display
from IPython.display import Image 
from IPython.core.display import HTML 

pd.options.display.max_columns = None
pd.options.display.max_rows=None


%matplotlib inline

### Import Data

In [2]:
##Using all cities/tows
#all_data = pd.read_csv('https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/all_data_transformed.csv', index_col=0)
##Using largest 300 cities/towns
all_data = pd.read_csv('https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/allDataCombined.csv', index_col=0)
print(all_data.shape)

(624, 639)


In [3]:
all_data = all_data[all_data['Population']>130]
all_data = all_data.reset_index(drop=True)

In [4]:
all_data.shape

(303, 639)

### Linear Model for Weights

___

For this sections we will use a simple Linear Regression model on all the dataset to weigh all the features included in the dataframe. We will regress on median household income.

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge, LinearRegression

In [6]:
target = all_data[['Median Household Income']]
target = MinMaxScaler().fit_transform(target)

predictors = all_data.drop(['City', 'Median Household Income','Homeownership','lat','lng'], axis=1)
X = MinMaxScaler().fit_transform(predictors)

In [7]:
reg = Ridge().fit(X, target)
reg.score(X, target)

0.9642832887195172

In [8]:
xxx = [i for i in reg.coef_]
xxx[0][1] = xxx[0][1]/5

In [9]:
coefficients = xxx
X = X * coefficients

Awesome! Now we have a normalized dataframe weighted based on the coefficients of our Linear Regression model!

---

Lets bring this back to a dataframe format.

In [10]:
norm_df = pd.DataFrame(X)
colNames = list(predictors.columns)

names_norm = {}

for i in range(0,len(colNames)):
    names_norm[i]=colNames[i]
    
norm_df = norm_df.rename(columns=names_norm)
norm_df.insert(0, 'City', all_data.City)

#Make sure to delete duplicates, just in case
norm_df.drop_duplicates('City', inplace=True)
print(norm_df.shape)
medval = norm_df['Median Property Value'] #Why? Because of bias
norm_df.drop('Median Property Value',1)
norm_df['Median Property Value'] = medval

# homie = norm_df['Homeownership']
# norm_df.drop('Homeownership',1)
# norm_df['Homeownership']=homie


pop = (norm_df['Population'])*1.5
norm_df.drop('Population',1)
norm_df['Population']=pop

norm_df.head()

(303, 635)


,City,Population,Median Property Value,Median Age,Commute Time,Health Care,Safety,Pollution,Summer Temp,Fall Temp,Winter Temp,Spring Temp,Summer Swing,Fall Swing,Winter Swing,Spring Swing,Summer Precip,Fall Precip,Winter Precip,Spring Precip,Summer Snow,Fall Snow,Winter Snow,Spring Snow,Zoo Exhibit,ATM,Accessories Store,Acupuncturist,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,Airport Tram,Alternative Healer,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Apres Ski Bar,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bathing Area,Bavarian Restaurant,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Bulgarian Restaurant,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Capitol Building,Car Wash,Caribbean Restaurant,Carpet Store,Casino,Castle,Cave,Cemetery,Cheese Shop,Child Care Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Cidery,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Baseball Diamond,College Basketball Court,College Bookstore,College Cafeteria,College Communications Building,College Football Field,College Gym,College Hockey Rink,College Library,College Quad,College Rec Center,College Residence Hall,College Soccer Field,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Country Dance Club,Coworking Space,Credit Union,Creperie,Cricket Ground,Cruise,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Distribution Center,Dive Bar,Dive Shop,Dive Spot,Doctor's Office,Dog Run,Donburi Restaurant,Doner Restaurant,Donut Shop,Drive-in Theater,Drugstore,Dry Cleaner,Dumpling Restaurant,Dutch Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Elementary School,Emergency Room,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fire Station,Fireworks Store,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Rest